In [1]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  libgfortran5
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 589 kB of archives.
After this operation, 2,660 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libgfortran5 amd64 8.4.0-1ubuntu1~18.04 [589 kB]
Fetched 589 kB in 1s (412 kB/s)
Selecting previously unselected package libgfortran5:amd64.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../libgfortran5_8.4.0-1ubuntu1~18.04_amd64.deb ...
Unpacking libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Setting up libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [2]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 4.7 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 


##1)


Let position of lake end be denoted by $(x_l, y_l)$\
Let position of farm end be denoted by $(x_f, y_f)$

Shreyas will want to minimize total length of the pipe. This length is equal to distance between ends of the pipe.
hence, total length is 
$ \sqrt{(x_l - x_f)^2 + (y_l - y_f)^2}$

if we consider his house to be at (0,0), his farm can be represented as the area bounded by the 4 lines:

$\begin{align}
x + 3y &= 0,\\
x - y &= 0,\\
x - y - 200 &= 0,\\
x + y + 200 &= 0,
\end{align}$
\
And the lake can be denoted by the eqn:
$(x - 200)^2 + (y - 200)^2 = 2500$

The constraints are that  $(x_f, y_f)$ should fall in the bounded region, and  $(x_l, y_l)$ should lie in the lake region

Decision variables: $x_l, y_l, x_f,y_f$

Objective: $\text{Minimize   } \sqrt{(x_l - x_f)^2 + (y_l - y_f)^2}$\
S.T\
$\begin{align}
x_f + 3y_f &\leq 0,\\
x_f - y_f &\geq 0,\\
x_f - y_f - 200 &\leq 0,\\
x_f + y_f + 200 &\geq 0,\\
(x_l - 200)^2 + (y_l - 200)^2 &\leq 2500,
\end{align}$

In [3]:
import numpy as np
from pyomo.environ import *

In [4]:
m2 = ConcreteModel()
m2.xl = Var()
m2.yl = Var()
m2.xf = Var()
m2.yf = Var()

m2.c = ConstraintList()
m2.c.add(expr= m2.xf + 3*m2.yf <= 0)
m2.c.add(expr= m2.xf - m2.yf >= 0)
m2.c.add(expr= m2.xf - m2.yf - 200 <= 0)
m2.c.add(expr= m2.xf + m2.yf + 200 >= 0)
m2.c.add(expr= (m2.xl - 200)**2 + (m2.yl - 200)**2 <= 2500)

m2.obj = Objective(expr=sqrt((m2.xl-m2.xf)**2 + (m2.yl - m2.yf)**2+1e-8))
#adding small epsilon e-8 to avoid chance of 0 coming in square root expression while iterating as this would make the derivative at that point undefined

In [5]:
solver = SolverFactory('ipopt')
res1 = solver.solve(m2)

In [6]:
print('\nDecision Variables')
print('xl = ', m2.xl.value)  
print('yl = ', m2.yl.value) 
print('xf = ', m2.xf.value)  
print('yf = ', m2.yf.value) 


Decision Variables
xl =  184.18861161834127
yl =  152.56583486353156
xf =  119.9999999872941
yf =  -39.999999995072876


In [7]:
print('The minimum length of pipe is ',m2.obj())

The minimum length of pipe is  202.98221256534453


##3)

One end of the pipe should be placed 40m south and 120m east of his house, and the other end should be placed 184.18m east and 152.56m north of this house.

##4)

Let position of junction be denoted by $(x_j, y_j)$\
Let position of lake end be denoted by $(x_l, y_l)$\
Let position of farm end be denoted by $(x_f, y_f)$

Shreyas will want to minimize total cost to set up the pipe. This cost is proportional to length of each part of the pipe. if we assume 1 m of regular pipe costs 1 unit, then 1m of the storehouse pipe will be 0.2 units.
hence, total cost is 
$ \sqrt{(x_l - x_j)^2 + (y_l - y_j)^2} + \sqrt{(x_j - x_f)^2 + (y_j - y_f)^2} +0.2\sqrt{(15 - x_f)^2 + (20 - y_f)^2}$

The constraints are same as in Q1

Problem:

Decision variables: $x_l, y_l, x_f,y_f, x_j, y_j$

Objective: $\text{Minimize   } \sqrt{(x_l - x_f)^2 + (y_l - y_f)^2} + \sqrt{(x_l - x_f)^2 + (y_l - y_f)^2} +0.2\sqrt{(x_l - x_f)^2 + (y_l - y_f)^2}$\
S.T\
$\begin{align}
x_f + 3y_f &\leq 0,\\
x_f - y_f &\geq 0,\\
x_f - y_f - 200 &\leq 0,\\
x_f + y_f + 200 &\geq 0,\\
(x_l - 200)^2 + (y_l - 200)^2 &\leq 2500,
\end{align}$

In [8]:
m21 = ConcreteModel()

In [9]:
m21.xl = Var()
m21.yl = Var()
m21.xf = Var()
m21.yf = Var()
m21.xj = Var()
m21.yj = Var()


In [10]:
m21.c = ConstraintList()
m21.c.add(expr= m21.xf + 3*m21.yf <= 0)
m21.c.add(expr= m21.xf - m21.yf >= 0)
m21.c.add(expr= m21.xf - m21.yf - 200 <= 0)
m21.c.add(expr= m21.xf + m21.yf + 200 >= 0)

m21.c.add(expr= (m21.xl - 200)**2 + (m21.yl -200)**2 <= 2500)


m21.obj = Objective(expr=sqrt((m21.xl-m21.xj)**2 + (m21.yl - m21.yj)**2 + 1e-8) + sqrt((m21.xf-m21.xj)**2 + (m21.yf - m21.yj)**2+1e-8) + 0.2*sqrt((15-m21.xj)**2 + 0.2*(20-m21.yj)**2 +1e-8))
#adding small epsilon e-8 to avoid chance of 0 coming in square root expression while iterating as this would make the derivative at that point undefined

In [11]:
res3 = solver.solve(m21)
#res3.write()

In [12]:
print('\nDecision Variables')
print('xl = ', m21.xl.value)  
print('yl = ', m21.yl.value) 
print('xf = ', m21.xf.value)  
print('yf = ', m21.yf.value)
print('xj = ', m21.xj.value)  
print('yj = ', m21.yj.value)  


Decision Variables
xl =  175.51860079928036
yl =  156.40342768147465
xf =  73.86507362814709
yf =  -24.62169120883139
xj =  73.86516918555895
yj =  -24.621404536595765


##6)

The lake end should be 156.4m north and 175.5m east of the house. The farm end should be 73m east and 24.62m south of the house. The junction should be in the same location as the farm end.
(Practically speaking, the junction should be as close as possible to the the farm end)